In [1]:
## DEPENDENCIAS

%%capture output
# !pip install nodejs
!pip install ipywidgets
!pip install jupyterlab_widgets
!pip install widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

UsageError: Line magic function `%%capture` not found.


In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join


from src.SimilarityArtworks import *
from src.SimilarityUsers import *
from src.ArtworksMatrix import *
from src.UsersMatrix import *
from src.UsersClustering import *
from src.AverageUser import *

In [2]:
import inspect

def inner_classes_list(cls):
    return [cls_attribute.__name__ for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]
            # and issubclass(cls_attribute, SimilarityFunctionInterface)]

In [3]:
def elegirDatos(path, files):
  usersDropdown = widgets.Dropdown(
      options=files,
      value='Prado_users_emotions_OnePolarity.csv',
      description='Users:',
      disabled=False,
  )
  itemsDropdown = widgets.Dropdown(
      options=files,
      value='Prado_artworks_wikidata.csv',
      description='Items:',
      disabled=False,
  )
  return usersDropdown, itemsDropdown

def cargarDatos(path, usersDropdown, itemsDropdown):
  users = pd.read_csv(path + usersDropdown.value, index_col=0)
  artworks = pd.read_csv(path + itemsDropdown.value)
  users['positive'] = users['positive'].apply(eval)
  users['negative'] = users['negative'].apply(eval)
  users['mixed'] = users['mixed'].apply(eval)
  return users, artworks

def elegirSimilitudItems(path, files, artworks):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityArtworks))

  dropdowns = {}
  for col in artworks.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)

  colorDropdown = widgets.Dropdown(options=files, value=None, description='Colors', disabled=False)

  return dropdowns, colorDropdown

def elegirSimilitudUsers(path, files, users):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityUsers))

  dropdowns = {}
  for col in users.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)
  
  return dropdowns

def elegirPesosItems(atr_dropdowns, colorDropdown):
  floatTexts = {}
  colorFloatText = None
  for key, value in atr_dropdowns.items():
    if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False, step = 0.1)

  if colorDropdown.value is not None:
    colorFloatText = widgets.FloatText(value=0.3, description='Colors', disabled=False,step = 0.1)
    
  return floatTexts, colorFloatText

def elegirPesosUsers(atr_dropdowns):
  floatTexts = {}
  for key, value in atr_dropdowns.items():
    if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False, step = 0.1)
  
  demographicFloatText = widgets.FloatText(value=0.5, description='Demographic', disabled=False, step = 0.1)
  polarityFloatText = widgets.FloatText(value=0.5, description='Polarity', disabled=False, step = 0.1)
  
  return floatTexts, demographicFloatText, polarityFloatText

def calcularSimilitudItems(path, artworks, atr_dropdowns, float_texts):
  funcs = {x : None for x in artworks.columns}
  weights = {x : None for x in artworks.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})
  weights.update({'Colors': colorFloatText.value})

  AM = ArtworksMatrix(artworks, function_dict=funcs, weight_dict=weights, colors_path=path + colorDropdown.value)
  
  return AM.getSimilarityMatrix()

def calcularSimilitudUsers(path, users, artworks_sim, atr_dropdowns, float_texts, demographicFloatText, polarityFloatText):
  funcs = {x : None for x in users.columns}
  weights = {x : None for x in users.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})

  weights.update({'polarity' : polarityFloatText.value, 'demographic' : demographicFloatText.value})  
  
  UM = UsersMatrix(users, artworks_sim, function_dict=funcs, weight_dict=weights)
  
  return UM.getSimilarityMatrix()

def calcularClusters(users, users_sim):
  users_distance = 1 - users_sim
  labels = UsersClustering(users_sim).kMedoidsFromMatrix() ## DISTANCES (?)
  users_clustered = users.copy()
  users_clustered['cluster'] = labels
  return users_clustered



In [4]:
data_path = 'data/'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

usersDropdown, itemsDropdown = elegirDatos(data_path, files)
display(itemsDropdown)
display(usersDropdown)

Dropdown(description='Items:', index=8, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.cs…

Dropdown(description='Users:', index=14, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.c…

In [5]:
users, artworks = cargarDatos(data_path, usersDropdown, itemsDropdown)
atr_dropdowns, colorDropdown = elegirSimilitudItems(data_path, files, artworks)
for key, value in atr_dropdowns.items():
  display(value)
display(colorDropdown)

Dropdown(description='ID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), value…

Dropdown(description='Category', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'),…

Dropdown(description='Artist', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), v…

Dropdown(description='Title', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), va…

Dropdown(description='Year', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), val…

...

Dropdown(description='wd:paintingID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColo…

Dropdown(description='wd:paintingURL', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityCol…

Dropdown(description='wd:artistID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors…

Dropdown(description='wd:artistURL', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColor…

Dropdown(description='Colors', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

In [6]:
# atributes_artworks = [i for i in [value.value for key, value in atr_dropdowns.items()] if i]
atributes_artworks = [key for key,value in atr_dropdowns.items() if value is not None]

float_texts, colorFloatText = elegirPesosItems(atr_dropdowns, colorDropdown)
for key, value in float_texts.items():
  display(value)

if colorDropdown.value is not None:
  display(colorFloatText) 

FloatText(value=0.3, description='Category', step=0.1)

FloatText(value=0.3, description='Artist', step=0.1)

FloatText(value=0.3, description='Colors', step=0.1)

In [7]:
artworks_sim = calcularSimilitudItems(data_path, artworks, atr_dropdowns, float_texts)

In [8]:
artworks_sim

,57726f48edc2cb3880ba012f,57726e34edc2cb3880b62340,57726f56edc2cb3880ba7233,57727c95edc2cb3880e4d1d4,57727016edc2cb3880bceb88,57726f78edc2cb3880baa755,57726f4cedc2cb3880ba07ed,57726f76edc2cb3880baa425,57726f4fedc2cb3880ba0b6d,5772700eedc2cb3880bc836e,...,577271fdedc2cb3880c384cc,57727015edc2cb3880bceb28,57727071edc2cb3880be3213,57726e37edc2cb3880b628ee,57726f49edc2cb3880ba034f,577278d5edc2cb3880d94196,5772752eedc2cb3880cd9f38,57727c2eedc2cb3880e39405,577278cdedc2cb3880d9370c,5772700eedc2cb3880bc82ce
57726f48edc2cb3880ba012f,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57726e34edc2cb3880b62340,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57726f56edc2cb3880ba7233,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57727c95edc2cb3880e4d1d4,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.5,0.0,0.0,...,0.5,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0
57727016edc2cb3880bceb88,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,1.0
57726f78edc2cb3880baa755,0.0,0.0,0.0,0.5,0.0,1.0,0.0,1.0,0.0,0.0,...,0.5,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0
57726f4cedc2cb3880ba07ed,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57726f76edc2cb3880baa425,0.0,0.0,0.0,0.5,0.0,1.0,0.0,1.0,0.0,0.0,...,0.5,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0
57726f4fedc2cb3880ba0b6d,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5772700eedc2cb3880bc836e,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,1.0


In [9]:
atr_dropdowns = elegirSimilitudUsers(data_path, files, users)
for key, value in atr_dropdowns.items():
  display(value)

Dropdown(description='userId', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simil…

Dropdown(description='age', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Similari…

Dropdown(description='gender', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simil…

Dropdown(description='country', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simi…

Dropdown(description='positive', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Sim…

Dropdown(description='negative', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Sim…

Dropdown(description='mixed', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simila…

In [10]:
# atributes_users = [i for i in [value.value for key, value in atr_dropdowns.items()] if i]
atributes_users = [key for key,value in atr_dropdowns.items() if value is not None]
float_texts, demographicFloatText, polarityFloatText = elegirPesosUsers(atr_dropdowns)
display(demographicFloatText)
display(polarityFloatText)
for key, value in float_texts.items():
  display(value) 

FloatText(value=0.5, description='Demographic', step=0.1)

FloatText(value=0.5, description='Polarity', step=0.1)

FloatText(value=0.3, description='age', step=0.1)

FloatText(value=0.3, description='gender', step=0.1)

FloatText(value=0.3, description='positive', step=0.1)

FloatText(value=0.3, description='negative', step=0.1)

FloatText(value=0.3, description='mixed', step=0.1)

In [11]:
users_sim = calcularSimilitudUsers(data_path, users, artworks_sim, atr_dropdowns, float_texts, demographicFloatText, polarityFloatText)

In [102]:
%%capture output
users_clustered = calcularClusters(users, users_sim)

In [103]:
AU = AverageUser(users_clustered, artworks, atributes_users, atributes_artworks)

In [104]:
explicators = AU.computeAverageUser()
explicators

,userId,age,gender,country,positive,negative,mixed,cluster
0,expl307,45-54,male,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 5772700eedc2cb3880b...","[57727015edc2cb3880bceb08, 57726f59edc2cb3880b...",0
1,expl308,35-44,female,Spain,"[57726f59edc2cb3880ba76b3, 57727cbaedc2cb3880e...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...",1
2,expl309,35-44,female,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 577278d5edc2cb3880d...","[57727cbaedc2cb3880e561f7, 57727015edc2cb3880b...",3
3,expl310,45-54,female,Spain,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726e45edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f49edc2cb3880b...",4
4,expl311,55-64,female,Spain,"[57726f59edc2cb3880ba76b3, 57727c2eedc2cb3880e...","[57726f56edc2cb3880ba7233, 57727cbaedc2cb3880e...",[57727015edc2cb3880bceb08],5


In [105]:
AU.printExplanation()

------------------------
Cluster:  0
	Individuos:  71
	age :
		 45-54 ( 26.76056338028169 %)
	gender :
		 male ( 100.0 %)
	country :
		 Spain ( 40.845070422535215 %)
	--Top 3  mixed--
			Title:St. Rufina of Seville
			ID: 57727015edc2cb3880bceb08
			Category: Baroque
			Artist: Francisco de Zurbaran
			Title: St. Rufina of Seville
			Year: 1635-1640
			Image URL: https://uploads6.wikiart.org/images/francisco-zurbaran/st-rufina-of-seville-1640.jpg!Large.jpg
			Painting Info URL: https://www.wikiart.org/en/francisco-de-zurbaran/st-rufina-of-seville-1640
			Artist Info URL: https://www.wikiart.org/en/francisco-de-zurbaran
			Location: Museo del Prado, Madrid, Spain
			wd:paintingID: Q77992354
			wd:paintingURL: http://www.wikidata.org/entity/Q77992354
			wd:artistID: Q209615
			wd:artistURL: http://www.wikidata.org/entity/Q209615
			Title:The Straw Manikin
			ID: 57726f59edc2cb3880ba76b3
			Category: Romanticism
			Artist: Francisco Goya
			Title: The Straw Manikin
			Year: 1791-1792
			I